In [1]:
from pvlib.iotools import get_pvgis_hourly
import pandas as pd

latitude = 12.9716
longitude = 77.5946
# for bng

start, end = '2005-01-01', '2023-12-31'

data, meta = get_pvgis_hourly(
    latitude=latitude,
    longitude=longitude,
    start=start,
    end=end,
    pvcalculation=True,   # tells PVGIS to calculate PV system output
    peakpower=5.0,        # 5 kW PV system
    pvtechchoice='crystSi',  # crystalline silicon panel
    mountingplace='building'
)

data

,P,poa_direct,poa_sky_diffuse,poa_ground_diffuse,solar_elevation,temp_air,wind_speed,Int
time,,,,,,,,
2005-01-01 00:30:00+00:00,0.00,0.00,0.00,0.0,0.00,17.88,1.93,0
2005-01-01 01:30:00+00:00,40.65,1.35,18.87,0.0,3.23,17.88,2.00,0
2005-01-01 02:30:00+00:00,690.55,72.60,98.00,0.0,16.29,18.15,2.34,0
2005-01-01 03:30:00+00:00,1384.05,141.10,169.35,0.0,28.66,19.84,2.76,0
2005-01-01 04:30:00+00:00,2151.50,267.95,213.50,0.0,39.80,21.54,2.28,0
...,...,...,...,...,...,...,...,...
2023-12-31 19:30:00+00:00,0.00,0.00,0.00,0.0,0.00,17.51,2.48,0
2023-12-31 20:30:00+00:00,0.00,0.00,0.00,0.0,0.00,16.87,2.34,0
2023-12-31 21:30:00+00:00,0.00,0.00,0.00,0.0,0.00,16.45,2.21,0


In [2]:
import requests
import datetime
import pvlib

# 1) Location (Bangalore example)
lat, lon = 12.9716, 77.5946

# 2) Current time (UTC, rounded to nearest hour)
now = datetime.datetime.now(datetime.UTC).replace(minute=0, second=0, microsecond=0)
time_str = now.strftime("%Y-%m-%dT%H:00")   # format like "2025-09-12T08:00" .

# 3) Get weather forecast from API
url = f"https://api.open-meteo.com/v1/forecast?latitude={lat}&longitude={lon}" \
      "&hourly=temperature_2m,windspeed_10m,direct_radiation,diffuse_radiation"
data = requests.get(url).json()   # request + convert to dictionary

# If no hourly data, stop
if "hourly" not in data:
    raise RuntimeError(f"No hourly data! Reason: {data.get('reason','Unknown')}")

hourly = data["hourly"]

# 4) Extract values for current hour
if time_str not in hourly["time"]:
    raise RuntimeError(f"Time {time_str} not found in API data!")

i = hourly["time"].index(time_str)   # find index of current hour

dni = float(hourly["direct_radiation"][i])   # Direct normal irradiance
dhi = float(hourly["diffuse_radiation"][i])  # Diffuse horizontal irradiance
ghi = dni + dhi                              # Approximate global irradiance
temp_air = float(hourly["temperature_2m"][i])  # Air temp (°C)
wind_speed = float(hourly["windspeed_10m"][i]) # Wind speed (m/s)

# 5) Solar position
solpos = pvlib.solarposition.get_solarposition(now, lat, lon)
solar_elev = float(solpos["elevation"].iloc[0])  # Sun height in degrees

# 6) Plane-of-array irradiance (for tilted panel)
tilt, azim = 30, 180  # 30° tilt, facing south
poa = pvlib.irradiance.get_total_irradiance(
    surface_tilt=tilt,
    surface_azimuth=azim,
    solar_zenith=90 - solar_elev,   # zenith = 90 - elevation
    solar_azimuth=solpos["azimuth"].iloc[0],
    dni=dni, ghi=ghi, dhi=dhi
)

# Break down POA (plane-of-array) components
poa_direct = float(poa["poa_direct"])
poa_sky_diffuse = float(poa["poa_sky_diffuse"])
poa_ground_diffuse = float(poa["poa_ground_diffuse"])

# 7) Collect results
data_entry = {
    "poa_direct": poa_direct,
    "poa_sky_diffuse": poa_sky_diffuse,
    "poa_ground_diffuse": poa_ground_diffuse,
    "solar_elevation": solar_elev,
    "temp_air": temp_air,
    "wind_speed": wind_speed,
    "Int": ghi   # Intensity (GHI)
}

# 8) Print nicely
print("Current compatible data entry:")
for key, val in data_entry.items():
    print(f"{key}: {val}")


Current compatible data entry:
poa_direct: 105.59927694263877
poa_sky_diffuse: 168.87529904249172
poa_ground_diffuse: 7.402096440909761
solar_elevation: 26.658132366594565
temp_air: 26.3
wind_speed: 9.9
Int: 442.0


In [3]:
meta

{'inputs': {'location': {'latitude': 12.9716,
   'longitude': 77.5946,
   'elevation': 911.0},
  'meteo_data': {'radiation_db': 'PVGIS-ERA5',
   'meteo_db': 'ERA5',
   'year_min': 2005,
   'year_max': 2023,
   'use_horizon': True,
   'horizon_db': None,
   'horizon_data': 'DEM-calculated'},
  'mounting_system': {'fixed': {'slope': {'value': 0, 'optimal': False},
    'azimuth': {'value': 0, 'optimal': False},
    'type': 'building-integrated'}},
  'pv_module': {'technology': 'c-Si', 'peak_power': 5.0, 'system_loss': 0.0},
  'descriptions': {'location': {'description': 'Selected location',
    'variables': {'latitude': {'description': 'Latitude',
      'units': 'decimal degree'},
     'longitude': {'description': 'Longitude', 'units': 'decimal degree'},
     'elevation': {'description': 'Elevation', 'units': 'm'}}},
   'meteo_data': {'description': 'Sources of meteorological data',
    'variables': {'radiation_db': {'description': 'Solar radiation database'},
     'meteo_db': {'descripti

In [4]:
import numpy as np
all_data = []

for lat in range(-90, 91, 1):       # 1° grid
    for lon in range(-180, 181, 1):
        try:
            data1, meta = get_pvgis_hourly(
                latitude=lat,
                longitude=lon,
                start="2020-01-01",
                end="2020-02-21",
                pvcalculation=True,
                peakpower=5.0,
                pvtechchoice="crystSi",
                mountingplace="building"
            )
            data1["latitude"] = lat
            data1["longitude"] = lon
            all_data.append(data1)
        except Exception as e:
            # If PVGIS fails here → skip ocean or invalid point
            pass

# Combine all datasets
global_df = pd.concat(all_data, axis=0, ignore_index=True)



KeyboardInterrupt



In [ ]:
from global_land_mask import globe
globe.is_land(-88 , 113)  # True for Bengaluru

In [ ]:
global_df